# Detecção automática de expressões multi-palavras

Este notebook mostra como detectar automaticamente frases comuns (aka multi-word expressions, word n-gram collocations) a partir de um stream de sentenças.

Paper interessante sobre o assunto: 
[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Os dados utilizados são os atos normativos do TCU que foram pré-processados utilizando a seguinte análise:

lower case, remoção de acentos, stemming


## Inicialização

In [1]:
DOCS_FOLDER = 'Dados/atos-processados/'
BIGRAM_MODEL='bigram_model.bin'
TRIGRAM_MODEL='trigram_model.bin'

## Obtenção dos dados

In [2]:
import glob
import os

def find_files(folder, remove_empty = False):
    """
    Find all files in [folder]

    folder  :   string
                    folder to search (not recursive)
    """
    files =  glob.iglob(folder + '**/*.txt', recursive=True)
    return files

In [3]:
def get_sentences(input_file_pointer):
    sentences = []
    while True:
        line = input_file_pointer.readline()
        
        if not line:
            break
        
        processed_line = line.strip()
        
        if processed_line:
            sentences.append(processed_line)
        
    return sentences

In [4]:
# Diretório deste notebook
diretorio_notebook = os.getcwd()

# Diretório onde estão armazenados os dados
diretorio_dados = os.path.join(diretorio_notebook, os.path.pardir, DOCS_FOLDER)

files = find_files(diretorio_dados)

atos = []

for i, fpath in enumerate(files):
    with open(fpath, encoding="utf8") as f:
        sentences = get_sentences(f)
        atos.append(sentences)

In [5]:
def date_diff_in_seconds(dt2, dt1):
    timedelta = dt2 - dt1
    return timedelta.days * 24 * 3600 + timedelta.seconds

In [6]:
from nltk.tokenize import word_tokenize
from datetime import datetime

sentence_stream = []

start = datetime.now()

for content in atos:
    sentenca_tokenizada = []
    for sentenca in content:
        sentenca_tokenizada = word_tokenize(sentenca)
        sentence_stream.append(sentenca_tokenizada)
        
end = datetime.now()
delta = date_diff_in_seconds(end, start)
print('Leitura dos dados concluída (took %d seconds)' % (delta))

Leitura dos dados concluída (took 67 seconds)


# Treinamento do modelo

In [7]:
from gensim.models.phrases import Phrases, Phraser

biset = set()
triset = set()

start = datetime.now()

bigram = Phraser(Phrases(sentence_stream, min_count=5))
trigram = Phraser(Phrases(bigram[sentence_stream], min_count=5))

for sent in sentence_stream:
    bigrams_ = [b for b in bigram[sent] if b.count('_') == 1]
    trigrams_ = [t for t in trigram[bigram[sent]] if t.count('_') == 2]

    biset.update(bigrams_)
    triset.update(trigrams_)
    
end = datetime.now()
delta = date_diff_in_seconds(end, start)
print('Extração dos bigramas e trigramas concluída (took %d seconds)' % (delta))

C:\Users\Alessandra\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Extração dos bigramas e trigramas concluída (took 56 seconds)


# Exemplos de bigramas e trigramas identificados

In [8]:
print(bigram['tribunal', 'cont', 'unia'])

['tribunal', 'cont_unia']


In [9]:
print(bigram['alessandra', 'requena'])

['alessandra', 'requena']


In [12]:
print(bigram['imov', 'rur'])

['imov_rur']


In [22]:
print(bigram['supriment','fund'])

['supriment_fund']


In [30]:
print(trigram[bigram['tribunal', 'cont', 'unia']])

['tribunal_cont_unia']


In [31]:
print(trigram[bigram['fund', 'particip', 'municipi']])

['fund_particip_municipi']


### Foram identificados 11719 bigramas: vejam exemplos

In [36]:
len(biset)

11719

Alguns bigramas interessantes:
* imóvel rural (imov_rur)
* gratificação natalina (gratific_natalin)
* incidente de uniformização (incident_uniformiz)

In [38]:
cont = 0
for idx, x in enumerate(biset):
    print(x)
    if idx > 100:
        break

riach_santan
incidente_redutor
estad_acr
sergip_sescoops
atest_obit
consanguine_afins
fortal_taboca
estadual_distrital
terceiriz_estagiari
compativ_dem
carat_experimental
melhor_pratic
diretori_municip
remoca_ped
licencapremi_assidu
decisa_irrecorr
civil_fnac
uberland_ufu
regia_df
formaca_red
fiscal_comercial
imov_rur
totaliz_imov
gram_loureir
assent_funcional
relat_subitemns
lanc_corret
eduard_duailib
rodovi_federalm
siti_dabad
gratific_natalin
prevenca_detecca
portador_necess
citaca_revel
infraestrutur_seinfraoper
financi_nord
itaun_sul
conceit_desempenh
georeferenci_passiv
obtem_font
pedr_osori
mat_queim
doutor_severian
homolog_estagi
eventu_diferenc
beneficiariotitul_vbmt
licenc_conced
olh_dagu
carnaubeir_penh
obrig_comet
amaral_ferrador
comprovad_estudant
capa_lea
sebastia_parais
condutor_veicul
quinz_minut
lago_velh
independente_objetiv
cbex_mptcu
dissemin_incentiv
jos_safira
vincul_ministerial
uj_rfb
vint_cent
pitagor_fas
relaca_accountability
diurn_escal
cons_cct
formulari_gpd


### Foram identificados 8641 trigramas: vejam exemplos

In [33]:
len(triset)

8641

Alguns trigramas interessantes:
* comitê olímpico brasileiro (comit_olimp_brasileir)
* redução de jornada de trabalho (reduca_jorn_trabalh)
* uso sustentável dos biomas (uso_sustent_biom)

In [10]:
cont = 0
for idx, x in enumerate(triset):
    print(x)
    if idx > 200:
        break

decisa_normativatcu_novembr
competent_indici_concernent
interposica_recurs_reconsider
port_hav_contrapart
jurisdica_propr_privat
bem_provel_orient
med_saneador_adot
unidad_integrant_secretariasger
fund_combat_erradic
superior_r_milha
ocupant_carg_agent
divers_respons_apresentas
engenheir_paul_frontin
fund_garant_safr
centr_inteligente_exercit
est_divid_dois
financeir_materi_human
aut_infraca_embarg
objetiv_correca_gramatical
dat_inici_vigenc
afirmat_context_uj
quinhent_mil_habitant
represent_adequad_univers
consentane_teor_respect
garant_capit_terceir
atual_precis_oportun
defensor_public_unia
declar_bens_rendiment
espirit_sant_suest
objet_convincent_construt
invest_caix_bi
coub_fundaca_institu
mesm_cess_convoc
fund_direit_crianc
outr_corespons_atos
servidor_carreir_tipic
cuj_pen_exced
apresent_defes_recolh
aca_ordin_preferenci
r_excetu_decorrent
energ_eletr_fge
referencial_basic_governanc
estatut_regiment_intern
jurisdicion_terceir_prolaca
imobiliz_patrimoni_liqu
organiz_atualiz_fichar